In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
# Load the stock prices CSV file (adjust the path as needed)
file_path = "stock_prices_5m.csv"
df = pd.read_csv(file_path)

# If a Date column exists, parse it as datetime and sort by time
if 'Datetime' in df.columns:
    df['Datetime'] = pd.to_datetime(df['Date'])

# Ensure sorting by Ticker symbol first, then by Date
df.sort_values(by=['Ticker', 'Datetime'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Data Processing
scaler = MinMaxScaler(feature_range=(0, 1)) 




KeyError: 'Date'

In [ ]:
def process_data(df, ticker, feature='Close', lookback=50):
    ticker_df = df[df['Ticker'] == ticker][['Date', feature]].copy()
    ticker_df.set_index('Date', inplace=True)
    scaled_data = scaler.fit_transform(ticker_df)
    
    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i, 0])
        y.append(scaled_data[i, 0])
    
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, y, scaler

In [ ]:
# Train-Test Split
def split_data(X, y, test_size=0.2):
    return train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:
# Model Training
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

In [ ]:
# Model Evaluation
def evaluate_model(model, X_train, y_train, X_test, y_test, epochs=50, batch_size=32):
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1)
    return history

In [ ]:
# Hyperparameter Tuning
def tune_hyperparameters():
    best_model = None
    best_loss = float("inf")
    best_params = None
    
    for lr in [0.01, 0.001, 0.0001]:
        for batch_size in [16, 32, 64]:
            model = build_lstm_model((50, 1))
            model.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')
            history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0)
            loss = min(history.history['val_loss'])
            
            if loss < best_loss:
                best_loss = loss
                best_model = model
                best_params = (lr, batch_size)
    
    return best_model, best_params

In [ ]:
# Prediction
def predict_future(model, X_test, scaler):
    predictions = model.predict(X_test)
    return scaler.inverse_transform(predictions.reshape(-1, 1))


In [ ]:
# Plot Results
def plot_predictions(actual, predicted):
    plt.figure(figsize=(10, 5))
    plt.plot(actual, color='blue', label='Actual Price')
    plt.plot(predicted, color='red', label='Predicted Price')
    plt.legend()
    plt.show()

In [ ]:
#Example Workflow
ticker_symbol = "AAPL"
X, y, scaler = process_data(df, ticker_symbol)
X_train, X_test, y_train, y_test = split_data(X, y)
model = build_lstm_model((X_train.shape[1], 1))
history = evaluate_model(model, X_train, y_train, X_test, y_test)
best_model, best_params = tune_hyperparameters()
predictions = predict_future(best_model, X_test, scaler)
plot_predictions(y_test, predictions)

KeyError: "['Date'] not in index"